In [2]:
#Importing libaries
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

#Parameters setting
sc_fct = 1.1              #scaling factor      
min_neigh = 5             #minimum number of neighbours to identify an area as the specific object
min_size_f = (30,30)      #smallest face dimensions accepted
min_size_e = (10,10)      #smallest eye dimension accepted 

#Initialising color variables for leds (BGR format)
col_up=(0,0,0)
col_down=(0,0,0)
col_left=(0,0,0)
col_right=(0,0,0)

#Defining the function 'calc_min_max' which calculates the minimum and maximum value of gray that the input image has
#The function has as input parameters: the input image end its two dimensions
#The function's outputs are the minimum and maximum values of gray
def calc_min_max(matr,ew,eh):
    
    min_gray=255
    max_gray=0
    a=0
    b=0
    for i in range(0,eh):
        for j in range(0,ew):
            if (matr[i][j]>max_gray):
                max_gray=matr[i][j]
            if (matr[i][j]<min_gray):
                min_gray=matr[i][j]
    return min_gray,max_gray



#Defining a function which calculates x and y of the white pixels mean on the mask
#The 'calc_mean' function has as input parameters: the input image and the two dimensions of the image
#The function's outputs are the x and y coordinates of the mean white pixel point
def calc_mean(matr,ew,eh):
    cont=0
    a=0
    b=0
    for i in range(0,eh):
        for j in range(0,ew):
            if (matr[i][j]==255):
                a=a+i;
                b=b+j;
                cont=cont+1;
    if cont>0:
        c=float(a/cont)
        d=float(b/cont)
        return c,d
    else:
        return 0,0
    
#Defining a function which makes leds change color depending on the middle point position
#The 'leds_on' function has as input parameters: the x and y coordinates of the mean white pixel point 
#and the two dimensions of the image where the 'calc_mean' function is applied
#It doesn't return anything because the color variables are global, so they can be seen even outside the function
def leds_on(x_m,y_m,ew,eh):
    global col_up
    global col_down
    global col_left
    global col_right
    if (x_m>ew/2):
        col_down=(0,255,0)
        col_up=(0,0,0)
    else:
        col_up=(0,255,0)
        col_down=(0,0,0)
    if (y_m>eh/2):
        col_right=(0,255,0)
        col_left=(0,0,0)
    else:
        col_left=(0,255,0)
        col_right=(0,0,0)
        
#Loading the cascade classifier files  
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eyeCascade=cv2.CascadeClassifier("haarcascade_eye.xml")

#Capturing the video from the webcam and opening a window to display it
video_capture = cv2.VideoCapture(0)
cv2.namedWindow("EasYE TRACKING")

#Saving the window dimensions
x_dim=int(video_capture.get(3))
y_dim=int(video_capture.get(4))


while True:
    #Capturing frames from the video
    ret, frame = video_capture.read()
    
    #Converting the image from colour to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Using the cascade classifier to identify faces in the video
    faces = faceCascade.detectMultiScale(       
        gray,
        scaleFactor=sc_fct,
        minNeighbors=min_neigh,
        minSize=min_size_f
        )
    #detectMultiscale function returns a list of rectangles described by the upper left vertex and the two dimensions
    
    #Drawing rectangles recognised by the detectMultiscale function
    for (x, y, w, h) in faces:
        cv2.rectangle(frame,(x, y), (x+w, y+h), (0, 255, 0), 2)
        
        #Defining new regions of interest    
        roi_gray=gray[y:y+h/2,x:x+w]
        roi_color=frame[y:y+h/2,x:x+w]
        
        #Using the cascade classifier to identify eyes within the new ROI just defined
        eyes= eyeCascade.detectMultiScale(         
            roi_gray,
            scaleFactor=sc_fct,
            minNeighbors=min_neigh,
            minSize=min_size_e
            )
        
        #Drawing rectangles returned by the detectMultiscale function
        for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (255, 191, 0), 2)
                 
                #Defining new regions of interest
                roi_gray2=roi_gray[ey:ey+eh, ex:ex+ew]
                
                #Calling function to calculate the minimum and maximum value of gray in the image
                min_gray,max_gray=calc_min_max(roi_gray2,ew,eh)
                
                #Defining threshold values for image filtering (the value corresponds to a grey level)
                upper=(max_gray-min_gray)*15/100 + min_gray
                lower=0
                
                
                #Applying inRange function to pull out from the input image only the pixels that are included in this specific gray range 
                mask = cv2.inRange(roi_gray2, lower, upper)
                #The output is an image where the elements, which are in the range, are shown in white, while everything else il black.  
                
                if (mask==None):
                    continue
                else:
                    #Calling functions to calculate middle point of the mask and to change leds color
                    x_m,y_m=calc_mean(mask,ew,eh)
                    leds_on(x_m,y_m,ew,eh)
                    
                    if ((x_m!=0)and(y_m!=0)):
                        #Drawing leds on the output image 
                        cv2.circle(frame,(x_dim-70,30),20,col_up,-10)
                        cv2.circle(frame,(x_dim-30,70),20,col_right,-10)
                        cv2.circle(frame,(x_dim-70,110),20,col_down,-10)
                        cv2.circle(frame,(x_dim-110,70),20,col_left,-10)
                        #Showing the filtered image and the webcam video
                        cv2.imshow('mask',mask)
                        cv2.imshow("Face and eyes", frame)
                    
    #Breaking the while cicle when 'return' is pressed           
    if cv2.waitKey(1)==27:
        break
              
#Closing al windows and turning webcams off
video_capture.release()
cv2.destroyAllWindows()

C:\Users\giada\Anaconda3\lib\site-packages\ipykernel\__main__.py:115: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\giada\Anaconda3\lib\site-packages\ipykernel\__main__.py:116: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\giada\Anaconda3\lib\site-packages\ipykernel\__main__.py:145: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
